# A basic implementation of Echo State Network

**Welcome!** 👋
In this project, we aim to help you understand and build a basic version of an Echo state Network (ESN) and using it to forcast the future values of the *Mackey-Glass time series*, a well known chaotic system. Through this notebook, you will gain hands-on experience in applying a reservoir computing approach to a challenging time series prediction task.

## What You Will Learn

1. **Mackey-Glass Time Series:**  
   We will introduce the Mackey-Glass series, describing its generation process and its characteristic chaotic behavior. You will also learn why it's frequently used as a benchmark for forecasting models.

2. **Echo State Networks (ESNs):**  
   You will get a conceptual overview of ESNs: what the reservoir is, how we control its dynamics through the spectral radius and input scaling, and why we can train the output layer with a simple linear regression approach rather than backpropagation.

3. **Practical Steps to Implementation:**  
   - How to generate and prepare the Mackey-Glass data (normalization, train-test split).  
   - How to initialize and run the ESN.  
   - How to train the readout weights using ridge regression.  
   - How to visualize and evaluate the ESN's performance on unseen data.

## Why ESNs?

Echo State Networks offer a unique combination of simplicity and power. They avoid some complexities of traditional recurrent networks by fixing the recurrent weights and training only the output layer. This leads to:

- **Efficient Training:** No need for complex backpropagation through time.  
- **Rich Dynamics:** The reservoir's random, recurrent structure can model a wide range of temporal patterns.  
- **Flexibility:** ESNs can be applied to many sequence-related problems with minimal adjustments.

## The Goal

By the end of this notebook, you will have a working ESN applied to a challenging nonlinear time series. Beyond just code execution, you will deepen your understanding of the underlying concepts, empowering you to experiment with different parameter settings and datasets.

---




## Background on Echo State Networks

### What is Reservoir Computing?

Reservoir computing is a computational framework derived from several recurrent neural network models. It is a model suited for temporal or sequential data processing, like time-series prediction, speech recognition and system modelling. Generally, a reservoir computing system will consist of a reservoir for mapping inputs, and a readout for pattern analysis. Reservoir computing often uses randomly connected networks to capture complex dynamics.  
A good reservoir system should be able to separate inputs. This is called separability condition.

### Key Concepts in Echo State Networks (ESNs)

1. **Reservoir:**  
   The reservoir is a large set of interconnected neurons with fixed and non-trainable random weights. When you feed time series data into the reservoir, it produces complex, nonlinear responses over time. This internal state can be thought of as a dynamic memory that retains information about recent inputs.

2. **Spectral Radius:**  
   The spectral radius is the largest absolute value of the reservoir's weight matrix eigenvalues. It controls the reservoir's stability and how signals propagate and fade over time. Typically, it's set to a value slightly less than 1 to ensure that the echo states eventually die out and don't explode. To make sense of this, consider a body of water like a pond, when a pebble is dropped into the water it produces ripples that eventually get weaker and disappear. This is what we try to simulate

3. **Input Weights:**  
   The input weights determine how external inputs drive the reservoir. Scaling these appropriately ensures the reservoir receives a meaningful range of values, neither saturating nor ignoring the input.

4. **Output (Readout) Weights:**  
   After passing the input through the reservoir, we train a set of linear output weights to map the reservoir's state to the desired target output. Because this is a simple linear regression problem, it's easy and fast to solve without backpropagating through the entire network.

### Why ESNs?

ESNs are popular for time series forecasting because they simplify training. Since you only adjust the readout weights, you avoid complex backpropagation through time. This efficiency is especially appealing for long sequences or chaotic data. ESNs can capture rich temporal patterns thanks to their large, random reservoir, and they often perform well on challenging nonlinear forecasting tasks.

### Key Parameters to Consider

- **Reservoir Size:**  
  The number of neurons in the reservoir. A larger reservoir might capture more complex dynamics but will be more computationally expensive.

- **Spectral Radius:**  
  Typically chosen slightly less than 1.0 to maintain the echo state property and stable dynamics.

- **Leaking Rate:**  
  Controls how quickly the reservoir's state is updated. A lower leaking rate slows down state changes, providing a longer “memory” of the past.

- **Regularization:**  
  When training the readout weights, adding ridge regression (L2 regularization) can prevent overfitting and improve generalization.

## Mackey-Glass Time Series

The Mackey-Glass time series comes from a mathematical equation that defines how certain quantities, like blood cells, change over time with a delay. Basically, it is able to model a system whose values depend on the current values as well as those that occurred a while ago. THis gives it a "memory" effect which makes it look chaotic even though it depends on a fixed equation

**Why is it used as a benchmark?**  

The Mackey-Glass series looks irregular and can be hard to predict which makes it an excellent test for evaluating forecasting methods. If a model can handle the Mackey-Glass series, it's likely capable of dealing with other challenging, real-world sequences.

---

## Environment Setup

Before we begin, let's make sure we have all the required dependencies installed. For this tutorial, we will be using:

- **NumPy:** For numerical computations and handling arrays.
- **Matplotlib:** For plotting the time series and visualization of predictions.

If you are running this notebook locally and don’t have these packages, you can install them directly from within the notebook. If you are running on an environment like Google Colab or a pre-configured environment, these should already be available.

Run the cell below if you need to install or upgrade any of these packages.


In [1]:
%pip install numpy matplotlib


## Generate and Prepare the Mackey-Glass Data

In this section, we will:

1. **Generate the Mackey-Glass time series:**  
   We will use a function to approximate the Mackey-Glass differential equation and produce a series of values.  
   
2. **Normalize the data:**  
   Normalization ensures that the input values are on a similar scale, which helps stabilize the training and improves the performance of the ESN.

3. **Split into Training and Test Sets:**  
   We will use the first portion of the data for training the ESN and the remaining portion for testing the model’s forecasting ability.



In [1]:
import numpy as np

def generate_mackey_glass(beta=0.2, gamma=0.1, tau=17, n_samples=3000, delta_t=1.0, seed=42):
    """
    Generate Mackey-Glass time series using a discrete approximation.
    Parameters
    ----------
    beta, gamma : float
        Parameters of the Mackey-Glass equation.
        beta controls how strongly the delayed input contributes to the system's growth.
        gamma controls how quickly the current value of x decays or is lost over time
    tau : int
        Time delay paramter. How long it takes for the past state to influence the current state
    n_samples : int
        Length of the time series to generate.
    delta_t : float
        Time step increment for the approximation.
    seed : int
        Random seed for reproducibility.
    
    Returns
    -------
    x : np.ndarray
        Generated Mackey-Glass time series of length n_samples.
    """
    
    np.random.seed(42)
    # We need a history array of length n_samples+tau to allow for delayed referencing
    x = np.zeros(n_samples + tau)
    # Initialize with some random values to start (you can also initialize with a constant)
    # We do this because the equation needs to have some "history"
    x[:tau] = 1.5 + 0.2 * np.random.randn(tau)
    # Implementing of the equation
    for t in range(tau, n_samples + tau - 1):
        x_tau = x[t - tau]
        dx = (beta * x_tau / (1 + x_tau**10)) - gamma * x[t]
        x[t+1] = x[t] + dx * delta_t

    # Return only the portion after the initial transient
    return x[tau:]

# Generate the Mackey-Glass series
data = generate_mackey_glass(n_samples=3000)

# Normalize the data
data_mean = np.mean(data)
data_std = np.std(data)
data = (data - data_mean) / data_std

# Split into train and test sets
train_fraction = 0.7
train_size = int(len(data)*train_fraction)
train_data = data[:train_size]
test_data = data[train_size:]

print("Train data length:", len(train_data))
print("Test data length:", len(test_data))


Train data length: 2100
Test data length: 900


## ESN Implementation

In this section, we will describe the key steps involved in setting up and using an Echo State Network:

1. **Initializing the Reservoir and Weights:**
   - We create a random recurrent weight matrix `W` for the reservoir and an input weight matrix `Win`.
   - The reservoir’s hidden states are not trained directly, only the output layer weights will be learned.

2. **Adjusting the Spectral Radius:**
   - We rescale the reservoir weight matrix `W` so that its largest eigenvalue (spectral radius) is slightly less than 1.
   - This ensures stability and the "echo state property," meaning that the internal states will not explode or vanish over time. (remember the pond analogy)

3. **Updating Reservoir States Given Input:**
   - At each time step, we feed the current input into the reservoir.
   - The reservoir state `x` is updated with a combination of the previous state, the new input, and the recurrent dynamics.
   - A “leaking rate” can control how fast the reservoir state changes.

4. **Collecting States for Training the Readout Layer:**
   - As we run the reservoir on the training data, we record all reservoir states.
   - We then use these recorded states (and the known target outputs) to solve a simple linear regression problem for the readout weights.

5. **Using Ridge Regression to Compute Readout Weights:**
   - Once we have a matrix of collected states and a vector of training targets, we use ridge regression (linear regression with L2 regularization) to find `Wout`.
   - `Wout` is the set of weights from the reservoir states to the output, and it is the only part of the ESN that gets trained.

Once the ESN is set up and `Wout` is computed, the network is ready to make predictions: it can generate outputs for new inputs by just updating the reservoir states and applying `Wout`.


In [1]:
import numpy as np
from numpy.linalg import pinv

# ESN Hyperparameters
input_dim = 1
reservoir_size = 200
output_dim = 1
spectral_radius = 0.95
input_scaling = 0.1
leaking_rate = 1.0
ridge_reg = 1e-6  # Regularization parameter for ridge regression

# Initialize input and reservoir weights
Win = (np.random.rand(reservoir_size, input_dim) - 0.5) * 2 * input_scaling
W = np.random.rand(reservoir_size, reservoir_size) - 0.5

# Adjust spectral radius
rhoW = max(abs(np.linalg.eigvals(W)))
W *= (spectral_radius / rhoW)

# At this point, the ESN is initialized. There will be no output for this cell
# We haven't trained Wout yet, that will happen after we collect reservoir states on the training data. 
